# Imports

In [ ]:
import pickle
import os
import numpy as np
import yaml
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd

%matplotlib inline

In [ ]:
run_id = 6

In [ ]:
with open('ros_packages/px4_ros_extended/src_py/params.yaml') as info:
    info_dict = yaml.load(info, Loader=yaml.SafeLoader)
        
n_test = info_dict["test_window_reward"]
n_train = info_dict["train_window_reward"]

# Dataframe Creation

In [ ]:
with open("logs/log_" + str(run_id) + ".pkl", "rb") as file_pkl:
    data = pickle.load(file_pkl)
train = pd.Series(data[0])
test = pd.Series(data[1])

# Reward Visualization

In [ ]:
# Visualize the result
means = train.rolling(n_train, min_periods=10).mean()
stds = train.rolling(n_train, min_periods=10).std()
x = range(0, len(means))
figure(figsize=(20, 8), dpi=80)
plt.plot(x, means, 'or')
plt.plot(x, means, '-', color='gray')

plt.fill_between(x, means - stds, means + stds,
                 color='gray', alpha=0.2)
plt.xlim(0, len(means)+1);
plt.xticks(ticks=range(0, len(means), 500));

In [ ]:
means = test.rolling(n_test, min_periods=1).mean()
stds = test.rolling(n_test, min_periods=1).std()
x = range(0, len(means))
figure(figsize=(20, 8), dpi=80)
plt.plot(x, means, 'or')
plt.plot(x, means, '-', color='gray')

plt.fill_between(x, means - stds, means + stds,
                 color='gray', alpha=0.2)
plt.xlim(0, len(means)+1);
plt.xticks(ticks=range(0, len(means), 9), labels=range(500, int(len(means)/9)*500 + 500, 500));

In [ ]:
test.groupby(test.index // 9).mean()

# Test Results Visualization

In [ ]:
folder_path = "test_logs/" + str(run_id)
file_list_start = os.listdir(folder_path)

## Clean anomalies based on threshold on last height

In [ ]:
file_list = []
for file in file_list_start:
    with open(folder_path + "/" + file, "rb") as file_pkl:
        data = pickle.load(file_pkl)
    if len(data[1]) == 2 and np.abs(data[0]['z'][-1]) < 0.15:  # Threshold height, otherwise not landed (vz not trained)
        file_list.append(file)
    elif len(data[1]) == 3 and np.abs(data[0]['z'][-1]) < 0.50:  # Threshold height, otherwise not landed (vz trained)
        file_list.append(file)

## Statistics

In [ ]:
last_pos = []  # List of last positions
last_vel = []  # List of last velocities
for file in file_list:
    with open(folder_path + "/" + file, "rb") as file_pkl:
        data = pickle.load(file_pkl)
    last_pos.append([np.abs(data[0]['x'][-1]), np.abs(data[0]['y'][-1]), np.abs(data[0]['z'][-1])])
    if len(data[1]) == 2:
        last_vel.append([np.abs(data[1]['vx'][-1]), np.abs(data[1]['vy'][-1])])
    elif len(data[1]) == 3:
        last_vel.append([np.abs(data[1]['vx'][-1]), np.abs(data[1]['vy'][-1]), np.abs(data[1]['vz'][-1])])
last_pos = np.array(last_pos)
last_vel = np.array(last_vel)

success_cont = 0
for i in range(len(last_pos)):
    if last_pos[i][0] <= 0.3 and last_pos[i][1] <= 0.3:
        success_cont += 1
sr = 100 * success_cont / len(last_pos)

print("Success Rate: {}".format(sr))

print("Mean distance from target: ({}, {}, {})".format(last_pos[:, 0].mean(), last_pos[:, 1].mean(), last_pos[:, 2].mean()))
print("Std distance from target: ({}, {}, {})".format(last_pos[:, 0].std(), last_pos[:, 1].std(), last_pos[:, 2].std()))
print("Max distance from target: ({}, {}, {})".format(last_pos[:, 0].max(), last_pos[:, 1].max(), last_pos[:, 2].max()))
print("90% percentile distance from target: ({}, {}, {})".format(np.percentile(last_pos[:, 0], 90), np.percentile(last_pos[:, 2], 90), np.percentile(last_pos[:, 2], 90)))
print()
if len(last_vel[0])==2:
    print("Mean velocity: ({}, {})".format(last_vel[:, 0].mean(), last_vel[:, 1].mean()))
    print("Std velocity: ({}, {})".format(last_vel[:, 0].std(), last_vel[:, 1].std()))
    print("Max velocity: ({}, {})".format(last_vel[:, 0].max(), last_vel[:, 1].max()))
    print("90% percentile velocity: ({}, {})".format(np.percentile(last_vel[:, 0], 90), np.percentile(last_vel[:, 1], 90)))
elif len(last_vel[0])==3:
    print("Mean velocity: ({}, {}, {})".format(last_vel[:, 0].mean(), last_vel[:, 1].mean(), last_vel[:, 2].mean()))
    print("Std velocity: ({}, {}, {})".format(last_vel[:, 0].std(), last_vel[:, 1].std(), last_vel[:, 2].std()))
    print("Max velocity: ({}, {}, {})".format(last_vel[:, 0].max(), last_vel[:, 1].max(), last_vel[:, 2].max()))
    print("90% percentile velocity: ({}, {}, {})".format(np.percentile(last_vel[:, 0], 90), np.percentile(last_vel[:, 1], 90), np.percentile(last_vel[:, 2], 90)))

## Position & Velocity over time, single episode

In [ ]:
with open(folder_path + "/" + file_list[1], "rb") as file_pkl:
    data = pickle.load(file_pkl)
positions = pd.DataFrame(data[0])
velocities = pd.DataFrame(data[1])
velocities_ref = pd.DataFrame(data[2])
time = data[3]

In [ ]:
ax = velocities.plot(title="Velocity/Time", figsize=(10, 4));
plt.ylim(-1, 1);
plt.xlim(0, len(velocities)+1);
plt.grid();
ax.set_xlabel("Time (s)");
ax.set_ylabel("Velocity (m/s)");
v_t = round(len(velocities)/time)
vel_list = list(range(0, len(velocities), v_t))
time_list = list(range(0, round(time), 1))
if len(vel_list) > len(time_list):
    time_list = list(range(0, round(time)+1, 1))
elif len(time_list) > len(vel_list):
    vel_list = list(range(0, len(velocities)+v_t, v_t))
plt.xticks(ticks=vel_list , labels=time_list);

In [ ]:
ax = positions.plot(title="Position/Time", figsize=(10, 4));
plt.ylim(-3, 3);
plt.xlim(0, len(positions)+1);
plt.grid();
ax.set_xlabel("Time (s)");
ax.set_ylabel("Position (m)");
p_t = round(len(positions)/time)+1
pos_list = list(range(0, len(positions), v_t))
time_list = list(range(0, round(time), 1))
if len(pos_list) > len(time_list):
    time_list = list(range(0, round(time)+1, 1))
elif len(time_list) > len(pos_list):
    pos_list = list(range(0, len(positions)+v_t, v_t))
plt.xticks(ticks=pos_list , labels=time_list);

In [ ]:
ax = velocities_ref.plot(title="Velocity (action)/Time", figsize=(10, 4));
plt.ylim(-1, 1);
plt.xlim(0, len(velocities_ref)+1);
plt.grid();
ax.set_xlabel("Time (s)");
ax.set_ylabel("Velocity (action)(m/s)");
v_t = round(len(velocities_ref)/time)
vel_list = list(range(0, len(velocities_ref), v_t))
time_list = list(range(0, round(time), 1))
if len(vel_list) > len(time_list):
    time_list = list(range(0, round(time)+1, 1))
elif len(time_list) > len(vel_list):
    vel_list = list(range(0, len(velocities_ref)+v_t, v_t))
plt.xticks(ticks=vel_list , labels=time_list);